# JsonOutputParser

JSON(JavaScript Object Notation): 데이터를 저장하고 구조적으로 전달하기 위해 사용되는 경량의 데이터 포맷.

출력값이 JSON형태이다. LLM이 데이터를 정확하고 효율적으로 처리하여 사용자가 원하는 형태의 JSON을 생성하기 위해서는, 모델의 용량( 예) inteligence, 파라미터등)가 충분히 커야한다.

- 객체: {}로 둘러쌓인 키-값의 집합이다.
- 배열: []로 둘러싸인 값의 순서 있는 목록이다. 배열 내의 값은 쉼표로 구분된다.
```
{
    "name" : "Ho",
    "age" : 30,
    "is_student" = true,
    "skills" : ["Python","RAG","SLM"],
    "address": {
        "street" : "123 main st",
        "city" : "Anytown"
    }
}
```

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [4]:
# 3.5같은 경우는 잘 안나오는 경우도 있다고 한다.
model = ChatOpenAI(model_name="gpt-4o-mini")

### 원하는 구조 정의

In [6]:
#원하는 데이터 구조 정의
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="헤시태그 형식의 키워드 (2개 이상)")

In [10]:
# JsonOutputParser 를 사용하여 파서를 설정하고, 프롬프트 템플릿에 지시사항을 주입.

# 질의 생성
question = "지구 온난화의 심각성에 대해 알려주세요."

# 파서를 설정하고, 프롬프트 템플릿에 지시사항을 주입한다.
parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"title": "Description", "description": "\uc8fc\uc81c\uc5d0 \ub300\ud55c \uac04\uacb0\ud55c \uc124\uba85", "type": "string"}, "hashtags": {"title": "Hashtags", "description": "\ud5e4\uc2dc\ud0dc\uadf8 \ud615\uc2dd\uc758 \ud0a4\uc6cc\ub4dc (2\uac1c \uc774\uc0c1)", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [13]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당싱은 친절한 AI 어시스턴트 입니다. 질문에 대해 간결하게 답변해주세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}")
    ]
)
# 프롬프트에 parser를 채워준다.
prompt = prompt.partial(format_instructions = parser.get_format_instructions())
chain = prompt | model | parser
chain.invoke({"question": question})

{'description': '지구 온난화는 기후 변화의 주요 원인으로, 온실가스의 증가로 인해 지구 평균 기온이 상승하고 있습니다. 이는 극단적인 기상 현상, 해수면 상승, 생태계 파괴 등 다양한 문제를 초래하고 있습니다.',
 'hashtags': '#지구온난화 #기후변화'}

In [15]:
answer = chain.invoke({"question": question})

In [18]:
type(answer)

dict

In [22]:
print(answer["description"])
print(answer["hashtags"])

지구 온난화는 지구의 평균 기온이 상승하여 기후 변화, 해수면 상승, 생태계 파괴 등을 초래하는 심각한 문제입니다.
#지구온난화 #기후변화


# Pydantic 없이 사용하기
question 또는 prompt를 잘 사용하자!

In [32]:
question = "지구 온난화에 대해서 설명해 주세요. 온난화에 대한 설명은 'description'에, 관련 키워드는 'hashtags'에 넣어주세요."

parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI어시스턴트입니다. 질문에 간결하게 답변하세요."),
        ("user", 
        """
        #Format: {format_instructions}
        
        #Question: {question}
        """
        )
    ]
    
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser

response = chain.invoke({"question": question})
print(response)

{'description': '지구 온난화는 대기 중 온실가스 농도가 증가하여 지구의 평균 기온이 상승하는 현상입니다. 이는 주로 인간 활동, 특히 화석 연료의 연소와 삼림 파괴로 인해 발생합니다. 온난화는 기후 변화, 해수면 상승, 생태계 파괴 등 다양한 환경 문제를 초래합니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#지구환경']}


In [37]:
question = "지구 온난화에 대해서 설명해 주세요."

parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI어시스턴트입니다. 질문에 대한 설명은 'description'에, 관련 키워드는 'hashtags'에 넣어주세요."),
        ("user", 
        """
        #Format: {format_instructions}
        
        #Question: {question}
        """
        )
    ]
    
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser

response = chain.invoke({"question": question})
print(response)

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 온실가스의 증가에 의해 발생합니다. 이 온실가스는 대기 중에 열을 가두어 지구의 기온을 상승시키며, 주로 이산화탄소, 메탄, 아산화질소 등이 포함됩니다. 지구 온난화는 기후 변화, 해수면 상승, 극단적인 기상 현상 등을 초래하며, 생태계와 인간 사회에 심각한 영향을 미칠 수 있습니다. 이를 해결하기 위해서는 온실가스 배출 감소와 지속 가능한 에너지 사용이 중요합니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#지속가능성', '#환경보호']}


In [34]:
print(model.invoke("RAG에서 사용되는 parser에 대해서 알려줘"))

content='RAG( Retrieval-Augmented Generation) 모델은 정보 검색과 텍스트 생성을 결합한 자연어 처리 모델입니다. RAG는 주로 두 가지 주요 구성 요소로 이루어져 있습니다: 정보 검색기(retriever)와 텍스트 생성기(generator).\n\n이 모델에서 사용되는 파서는 주로 다음과 같은 역할을 수행합니다:\n\n1. **문서 검색**: 입력된 질문이나 쿼리에 대해 관련 문서를 검색합니다. 이 단계에서는 일반적으로 BM25와 같은 전통적인 정보 검색 기법이나, BERT와 같은 딥러닝 기반의 임베딩 방법을 사용하여 쿼리와 문서 간의 유사도를 계산합니다.\n\n2. **문서 처리**: 검색된 문서들을 파싱하여 필요한 정보만 추출합니다. 이 과정에서 문서의 구조를 이해하고, 필요한 데이터를 필터링하는 작업이 이루어질 수 있습니다.\n\n3. **텍스트 생성**: 검색된 문서에서 얻은 정보를 바탕으로 응답을 생성합니다. 이 단계에서는 Transformer 기반의 언어 모델이 사용되어, 자연스러운 언어로 질문에 대한 답변을 생성합니다.\n\nRAG 모델은 이러한 파서의 과정을 통해 사용자가 입력한 질문에 대해 더 정확하고 풍부한 정보를 제공할 수 있도록 돕습니다. 이를 통해 모델은 단순한 텍스트 생성에 그치지 않고, 외부의 지식 기반에서 정보를 가져와 보다 신뢰성 있는 답변을 생성하는 능력을 갖추게 됩니다. \n\n이와 같은 구조는 RAG 모델이 기존의 언어 생성 모델보다 정보의 정확성과 관련성을 높이는 데 기여합니다.' response_metadata={'token_usage': {'completion_tokens': 356, 'prompt_tokens': 17, 'total_tokens': 373}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0f03d4f0ee', 'finish_reason': 'stop', 'logprobs': None} id='r

In [35]:
print(model.invoke("RAG에서 사용되는 parser에 대한 정의 알려줘"))

content='RAG( Retrieval-Augmented Generation)에서 사용되는 파서는 주어진 질문이나 요청을 이해하고, 관련된 정보를 검색하기 위한 구조화된 데이터를 추출하는 역할을 합니다. RAG는 일반적으로 두 가지 주요 구성 요소로 이루어져 있습니다: 정보 검색기와 생성기.\n\n1. **정보 검색기 (Retriever)**: 사용자의 질문에 대한 관련 문서나 정보를 검색합니다. 이 단계에서는 대량의 데이터베이스나 문서에서 질문과 관련된 정보를 찾는 것이 중요합니다.\n\n2. **생성기 (Generator)**: 검색된 정보를 바탕으로 최종 응답을 생성합니다. 이 단계에서는 자연어 처리 모델을 사용하여 검색된 정보를 조합하고, 사용자가 이해할 수 있는 형태로 답변을 생성합니다.\n\n파서는 이 두 단계 간의 중간 역할을 하며, 검색된 정보를 적절히 처리하고, 생성기에게 전달하는 과정을 포함합니다. 따라서, RAG에서의 파서는 정보 검색과 자연어 생성 간의 효율적인 연결을 도와주는 중요한 역할을 합니다.\n\nRAG 시스템의 효율성과 정확성은 파서의 성능에 크게 의존하므로, 잘 설계된 파서는 시스템의 전반적인 성능을 향상시키는 데 기여합니다.' response_metadata={'token_usage': {'completion_tokens': 271, 'prompt_tokens': 18, 'total_tokens': 289}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b0abffe81', 'finish_reason': 'stop', 'logprobs': None} id='run-f4a32a37-9e22-4a91-826e-1b3a50a8a973-0' usage_metadata={'input_tokens': 18, 'output_tokens': 271, 'total_tokens': 289}
